In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels
import pickle
import graphviz
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('fivethirtyeight')

%matplotlib inline
from tqdm import tqdm
# modelling imports
from xgboost import XGBClassifier
from sklearn import svm, datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export_text, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix, f1_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
selected_columns = pickle.load( open("select_cols.p", "rb"))
selected_columns

In [ ]:
tw = selected_columns.sample(50000)
ttw = selected_columns.sample(1000)
tttw = selected_columns.sample(50)

- X = all vars
- Xs = all vars sampled (50,000) from total (1,828,069)
- Z = selected features
- Zs = sampled selected features

In [3]:
X, Xs, y, ys = selected_columns.drop(columns=['>50K', 'YRMARR', 'PWTYPE', 'BPLD', 'DEGFIELDD', 'EDUCD', 'PWMET13', 'BIRTHYR', 'SLWT', 'BPLD', 'RELATED', 'ANCESTR1D', 'EMPSTATD', 'PWMET13ERR', 'COUNTYICP']), tw.drop(columns=['>50K', 'YRMARR', 'PWTYPE', 'BPLD', 'DEGFIELDD', 'EDUCD', 'PWMET13', 'BIRTHYR', 'SLWT', 'BPLD', 'RELATED', 'ANCESTR1D', 'EMPSTATD', 'PWMET13ERR', 'COUNTYICP']), selected_columns.filter(['>50K']), tw.filter(['>50K'])

Z = selected_columns.filter(['EDUC', 'UHRSWORK', 'OCC', 'VALUEH', 'DEGFIELD', 'AGE', 'SEX', 'RACE', 'TRANWORK'])
Zs = tw.filter(['EDUC', 'UHRSWORK', 'OCC', 'VALUEH', 'DEGFIELD', 'AGE', 'SEX', 'RACE', 'TRANWORK'])

NameError: name 'selected_columns' is not defined

In [4]:
features = list(Z.columns)
features

NameError: name 'Z' is not defined

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

train_df = X_train.copy()
train_df['$$$'] = y_train

Z_train, Z_test, q_train, q_test = train_test_split(Z, y, test_size=0.2, random_state=42)
Zs_train, Zs_test, qs_train, qs_test = train_test_split(Zs, ys, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

In [6]:
std = StandardScaler()
X_train_scaled = std.fit_transform(X_train)
Xs_train_scaled = std.fit_transform(Xs_train)
Z_train_scaled = std.fit_transform(Z_train)
Zs_train_scaled = std.fit_transform(Zs_train)

NameError: name 'StandardScaler' is not defined

## KNN and confusion matrix

In [7]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(Xs, ys)
knn_confusion = confusion_matrix(ys_test, knn.predict(Xs_test))
plt.figure(dpi=100)
sns.heatmap(knn_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d')

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('kNN confusion matrix');

NameError: name 'KNeighborsClassifier' is not defined

## Logistic Regression

In [17]:
Cs = [1,10,100,1000,10000]
solvers = ['liblinear', 'newton-cg', 'sag', 'lbfgs']
f1_scores = []

for c in Cs:
    for solver in solvers:
        lr = LogisticRegression(C=c, solver=solver)
        lr.fit(Xs_train_scaled, ys_train)
        print('----------------------------------')
        print(f"C = {c}, Solver = {solver}")
        print(f'f1 = {f1_score(lr.predict(Xs_test), ys_test)}')
        print(f'r2 train= {lr.score(Xs_train_scaled, ys_train)}')
        print(f'r2 test= {lr.score(X_test, ys_train)}')
        print('----------------------------------')


/home/nick/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


----------------------------------
C = 1, Solver = liblinear
f1 = 0.012303906490310676
r2 train= 0.821725
r2 test= 0.6786310152237058
----------------------------------


/home/nick/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


----------------------------------
C = 1, Solver = newton-cg
f1 = 0.012300123001230014
r2 train= 0.82175
r2 test= 0.6786091342235253
----------------------------------


/home/nick/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

/home/nick/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


----------------------------------
C = 10000, Solver = newton-cg
f1 = 0.011083743842364532
r2 train= 0.821775
r2 test= 0.678636485473751
----------------------------------


/home/nick/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


----------------------------------
C = 10000, Solver = sag
f1 = 0.01290719114935464
r2 train= 0.821775
r2 test= 0.6785790478482772
----------------------------------
----------------------------------
C = 10000, Solver = lbfgs
f1 = 0.011083743842364532
r2 train= 0.821775
r2 test= 0.6786392205987736
----------------------------------


/home/nick/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


## Random Forest

In [15]:
min_samples_split = [2,4,6,10,20]


for num in min_samples_split:
    rf = RandomForestClassifier(min_samples_split=num)
    rf.fit(Xs_train_scaled, ys_train)
    print()

/home/nick/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


KeyboardInterrupt: 